In [1]:
import pandas as pd

In [2]:
file_path = "./Sample_input 2.xlsx"
input_data = pd.read_excel(file_path, sheet_name=0)
input_data['PRITITLE'].fillna('No Title', inplace=True)
input_data['Length'].fillna(0, inplace=True)

In [3]:
def calculate_indexname(indices):
    return 1 if 'S&P 500'  in indices.values or 'S&P 400' in indices.values else 0
def calculate_business_transactions(indices):
    #return the number of 'Yes' in indices
    return indices.value_counts().get('Yes', 0)

def calculate_attorney_presence(titles):
    return 1 if any('Attorney' in title for title in titles if isinstance(title, str)) else 0
def calculate_account_presence(titles):
    return 1 if any('Accountant' in title for title in titles if isinstance(title, str)) else 0
def calculate_ratio(group):
    sum_current_column = group['Directors aged 65 or above divided by board size'].sum()
    sum_board_size = group['Board Size'].sum()
    if sum_board_size == 0:
        return 0
    return sum_current_column / sum_board_size

In [7]:
# replace all instances of 'Null' in column Length with 0
input_data['Length'] = input_data['Length'].replace('Null', 0)
# convert the data type of column Length to numeric.
input_data['Length'] = pd.to_numeric(input_data['Length'], errors='coerce')

In [8]:
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225429 entries, 0 to 225428
Data columns (total 54 columns):
 #   Column                                                                 Non-Null Count   Dtype  
---  ------                                                                 --------------   -----  
 0   year                                                                   225429 non-null  int64  
 1   Ticker                                                                 224787 non-null  object 
 2   CUSIP                                                                  225429 non-null  object 
 3   Name                                                                   225429 non-null  object 
 4   Board Size                                                             225429 non-null  int64  
 5   Age                                                                    225171 non-null  float64
 6   Ethnicity                                                              22542

In [9]:
agg = {"Board Size":"sum", "Age":"mean","Ethnicity":"mean","Country_of_Empl":"mean","Classification/Independent":"mean","Outside_Public_Boards":"sum","Length":"mean","Num_Of_Shares":"sum","Pcnt_Ctrl_Votingpower":"sum","Nom_female member(1 if female is member)":"sum","CG_female member(1 if female is member)":"sum","Comp_female member(1 if female is member)":"sum","Audit_female member(1 if female is member)":"sum","CG_female member(1 if female is member)":"sum","Comp_female member(1 if female is member)":"sum","Audit_female member(1 if female is member)":"sum","Nom_female member(1 if female is member)":"sum","Prof_Services_YN":"sum",
       "indexname (1 if index equal to S&P 500 or less otherwise 0":lambda x : calculate_indexname(x),
       "PRITITLE" : lambda x: x.tolist(),
      'Directors aged 65 or above divided by board size': "sum" ,
      "No. of independent with aged 65 or above divided by total independent":"sum",
      "Classification/Independent":"mean",
      "Classification/Independent":"sum",
      "Audit_total member":"sum",
      'OWNLESS1':"sum",
      "Business_Transaction":lambda x : calculate_business_transactions(x),
      "OtherLink (if yes than 1)":lambda x : calculate_business_transactions(x),
      "Charity (if yes than 1)":lambda x : calculate_business_transactions(x),
      "Interlocking (if yes than 1)":lambda x : calculate_business_transactions(x),
      "Relative_yn (if yes than 1)":lambda x : calculate_business_transactions(x),
      'NON_CEO_LEADER (if yes than 1)':lambda x : calculate_business_transactions(x),
      'Former_employee_YN (if yes than 1)':lambda x : calculate_business_transactions(x),
      'Employment_Chairman (if yes than 1)':lambda x : calculate_business_transactions(x),
      'Employment_COO (if yes than 1)':lambda x : calculate_business_transactions(x),
      'Employment_CFO (if yes than 1)' : lambda x : calculate_business_transactions(x),
      'Employment_CEO (if yes than 1)':lambda x : calculate_business_transactions(x),
       'SUCC_COMM (if yes than 1)' : lambda x : calculate_business_transactions(x),
       'Attend_LESS75_PCT': lambda x : calculate_business_transactions(x),
       'Attend_LESS75_for independent':"sum",
       
       "Independent having length 15 or greater than 15":"sum",
       "Female":"sum","Financial_Expert":"sum",
       'CG_total member' : "sum","Comp_total member":"sum",
       'Nom_total member':"sum",'No. of Independent (1) having 3 or more than 3 outside public board':"sum",}
grouped = input_data.groupby(['Ticker', 'year','CUSIP','Name'])
# Apply aggregations
g = input_data.groupby(['Ticker', 'year','CUSIP','Name'])["Classification/Independent"].mean()
result = grouped.agg(agg).reset_index()



In [ ]:
result['Accountant'] = result['PRITITLE'].apply(calculate_account_presence) 
result['Attorney'] = result['PRITITLE'].apply(calculate_attorney_presence)
result["Directors aged 65 or above divided by board size"] = result["Directors aged 65 or above divided by board size"] / result["Board Size"]
result["No. of independent with aged 65 or above divided by total independent"] = result["No. of independent with aged 65 or above divided by total independent"] / result["Classification/Independent"]
result["Female"] = result["Female"]/result["Board Size"]
result["Financial_Expert"] = result["Financial_Expert"]/result["Board Size"]
result['Percentage of Independent (1) having more than 3 outside public board'] = result['No. of Independent (1) having 3 or more than 3 outside public board']/result["Board Size"]

result['Audit_membership'] = 0
result['CG_membership'] = 0
result['Comp_membership'] = 0
result["DirSince"] = pd.NA
result['Nom_membership'] = 0
result['Year_Term_Ends'] = pd.NA




In [ ]:
result["Classification/Independent"]=list(g)
result = result.reindex(columns = ['year', 'Ticker', 'CUSIP', 'Name', 'Board Size', 'Age', 'Ethnicity',
       'indexname (1 if index equal to S&P 500 or less otherwise 0',
       'PRITITLE', 'Attorney',
       'Accountant',
       'Country_of_Empl', 'Directors aged 65 or above divided by board size',
       'Classification/Independent',
       'No. of independent with aged 65 or above divided by total independent',
       'Outside_Public_Boards',
       'No. of Independent (1) having 3 or more than 3 outside public board',
       'Percentage of Independent (1) having more than 3 outside public board',
       'DirSince', 'Year_Term_Ends', 'Length',
       'Independent having length 15 or greater than 15', 'OWNLESS1',
       'Num_Of_Shares', 'Pcnt_Ctrl_Votingpower', 'Female',
       'Nom_female member(1 if female is member)',
       'CG_female member(1 if female is member)',
       'Comp_female member(1 if female is member)',
       'Audit_female member(1 if female is member)', 'Nom_membership',
       'CG_membership', 'Comp_membership', 'Audit_membership',
       'CG_total member', 'Nom_total member', 'Comp_total member',
       'Audit_total member', 'Employment_CEO (if yes than 1)',
       'Employment_Chairman (if yes than 1)', 'Employment_CFO (if yes than 1)',
       'Employment_COO (if yes than 1)', 'Attend_LESS75_PCT',
       'Attend_LESS75_for independent', 'Prof_Services_YN',
       'Former_employee_YN (if yes than 1)', 'Business_Transaction',
       'Relative_yn (if yes than 1)', 'Interlocking (if yes than 1)',
       'Charity (if yes than 1)', 'OtherLink (if yes than 1)',
       'Financial_Expert', 'SUCC_COMM (if yes than 1)',
       'NON_CEO_LEADER (if yes than 1)'])

In [ ]:
output_file = 'output_file2.xlsx'
result.to_excel(output_file, index=False)

In [ ]:
o  = pd.read_excel(".//output_file.xlsx", sheet_name=0)
oo = pd.read_excel(".//Sample_Output.xlsx", sheet_name=0)